In [ ]:
from typing import List

CATALOG = ""

SCHEMAS_TO_CLEAN: List[str] = [
    "aviacao_landing",
    "aviacao_bronze",
    "aviacao_meta",
]

META_SCHEMA = "aviacao_meta"
WM_TABLE_NAME = "watermark_incremental"

VOLUME_CATALOG = "main"
VOLUME_SCHEMA = "engenharia_dados"
VOLUME_NAME = "aviacao_landing"

LANDING_CSV_BASE_PATH = f"/Volumes/{VOLUME_CATALOG}/{VOLUME_SCHEMA}/{VOLUME_NAME}/aviacao/landing"


def qschema(schema: str) -> str:
    if CATALOG:
        return f"{CATALOG}.{schema}"
    return schema


def qtable(schema: str, table: str) -> str:
    if CATALOG:
        return f"{CATALOG}.{schema}.{table}"
    return f"{schema}.{table}"


def qname(schema: str, table: str) -> str:
    if CATALOG:
        return f"{CATALOG}.{schema}.{table}"
    return f"{schema}.{table}"


wm_full_name = qname(META_SCHEMA, WM_TABLE_NAME)
print(f"\n>>> Drop tabela de watermark (reset incremental): {wm_full_name}")
try:
    spark.sql(f"DROP TABLE IF EXISTS {wm_full_name}")
    print("Tabela de watermark removida. Próxima carga vai ler tudo de novo.")
except Exception as e:
    print(f"Erro ao dropar {wm_full_name}: {e}")


for schema in SCHEMAS_TO_CLEAN:
    schema_full = qschema(schema)
    print(f"\n=== Limpando tabelas do schema: {schema_full} ===")

    try:
        tables_df = spark.sql(f"SHOW TABLES IN {schema_full}")
    except Exception as e:
        print(f"Schema {schema_full} não existe ou sem permissão. Detalhe: {e}")
        continue

    for row in tables_df.collect():
        tbl_name = row["tableName"]
        fq_name = qtable(schema, tbl_name)
        print(f"DROP TABLE {fq_name}")
        spark.sql(f"DROP TABLE IF EXISTS {fq_name}")

for schema in SCHEMAS_TO_CLEAN:
    schema_full = qschema(schema)
    print(f"\nDROP SCHEMA {schema_full} CASCADE")
    try:
        spark.sql(f"DROP SCHEMA IF EXISTS {schema_full} CASCADE")
    except Exception as e:
        print(f"Erro ao dropar schema {schema_full}: {e}")


print(f"\nRemovendo arquivos da landing em: {LANDING_CSV_BASE_PATH}")
try:
    dbutils.fs.rm(LANDING_CSV_BASE_PATH, True)
    print("Arquivos CSV da landing removidos.")
except Exception as e:
    print(f"Não foi possível remover {LANDING_CSV_BASE_PATH}: {e}")

print("\n=== Ambiente de aviação zerado (dados, schemas e incremental) ===")
